In [1]:
!pip3 install torch
!pip3 install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 MB 4.4 MB/s eta 0:00:0000:0100:01m
  Using cached filelock-3.12.2-py3-none-any.whl (10 kB)
  Using cached typing_extensions-4.7.1-py3-none-any.whl (33 kB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 3.8 MB/s eta 0:00:0000:01
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 1.2 MB/s eta 0:00:0000:0100:010m
  Using cached kiwisolver-1.4.4-cp311-cp311-macosx_11_0_arm64.whl (63 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 4.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [135]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [136]:
names_list = open('names.txt','r').read().split('\n')

In [137]:
print(names_list[:10])

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']


In [138]:
char_to_int = {}
int_to_char = {}

char_to_int['.'] = 0
int_to_char[0] = '.'

for char_num in range(ord('a'),ord('z')+1):
    integer_representation = char_num-ord('a')+1
    char_to_int[chr(char_num)] = integer_representation
    int_to_char[integer_representation] = chr(char_num)


In [141]:
#hyperparameters

embed_size = 6

state_size = 20

hidden_size = 100

vocab_size = 28

In [142]:
import math
char_vector_size = 2
input_contexts , input_labels = [[] for _ in range(5)], [[] for _ in range(5)]

for name in names_list:
    log_padded_length = math.ceil(math.log(len(name))/math.log(2))
    padded_length = int(2**log_padded_length)
    context = [char_to_int[char] for char in name]  + [0] + [vocab_size-1]*(padded_length-len(name)) 
    input_contexts[log_padded_length].append(context)
#   instead of using labels, we can compare the result to the character vectors to find a likely match

In [143]:
freqs = [0 for i in range(vocab_size-1)]

for log_len in range(1,5):
    contexts = input_contexts[log_len]
    for c in contexts:
        for int_char in c:
            if int_char<vocab_size-1:
                freqs[int_char] += 1
                
freqs = torch.tensor(freqs+[0],dtype=torch.float64)
freqs/=freqs.sum()

freqs

tensor([0.1404, 0.1485, 0.0116, 0.0155, 0.0241, 0.0895, 0.0040, 0.0084, 0.0334,
        0.0776, 0.0127, 0.0221, 0.0612, 0.0291, 0.0803, 0.0348, 0.0045, 0.0012,
        0.0557, 0.0355, 0.0244, 0.0137, 0.0113, 0.0041, 0.0031, 0.0428, 0.0105,
        0.0000], dtype=torch.float64)

In [144]:
word = input_contexts[2][1]

In [145]:
# parameters
all_char_embeds = torch.randn((vocab_size,embed_size))

all_char_embeds[vocab_size-1] = 10**10

w_eh = torch.randn((embed_size, hidden_size)) * 0.5

w_sh = torch.randn((state_size, hidden_size)) * 0.5
w_so = torch.randn((state_size, embed_size)) *0.5

w_hs = torch.randn((hidden_size, state_size)) *0.5

b_h = torch.zeros((hidden_size,))
b_s = torch.zeros((state_size,))
b_o = torch.zeros((embed_size,))

init_state = 2*torch.sigmoid(torch.randn((state_size,)))

network_params = [init_state,w_eh,w_sh,w_so,w_hs,b_h,b_s,b_o,all_char_embeds]

for p in network_params:
    p.requires_grad = True

In [146]:
def process_state_and_embed(states , embeds, hiddens, outputs, idx):
    hiddens[idx] = 2*torch.sigmoid((states[idx] @ w_sh) + (embeds[idx] @ w_eh) + b_h) -1
    states[idx+1] = 2*torch.sigmoid(hiddens[idx] @ w_hs + b_s) - 1
    outputs[idx] = states[idx] @ w_so + b_o

In [147]:
batch_size = 32

for i in range(len(input_contexts)):
    input_contexts[i] = torch.tensor(input_contexts[i])

In [148]:
loss_tot = 0

loops = 100000
check_every = 500

# training loop
for i in range(loops):
    log_idx = random.randint(1,4)
#     log_idx = 4
    batch_indices = torch.randint(0,input_contexts[log_idx].shape[0],(batch_size,))

    batch = input_contexts[log_idx][batch_indices]

    word_length = len(batch[0])

    embeds = torch.transpose(all_char_embeds[batch],0,1)
    
    factors = torch.where(freqs > 0,1/(freqs+1e-18),0)
    
    factors[:-1] /= factors[:-1].mean()
    
    adjustment_factors = torch.transpose(factors[batch],0,1)
    
    adjustment_factors = adjustment_factors.view((word_length,batch_size,1))
    
    adjustment_factors[:-1] /= adjustment_factors[:-1].mean()
#     print(adjustment_factors , embeds.shape)

    states = [init_state] + [torch.zeros((batch_size,state_size,)) for _ in range(word_length-1)]

    hiddens = [torch.zeros((batch_size,embed_size,)) for _ in range(word_length-1)]

    outputs = torch.zeros((word_length,batch_size,embed_size))

    for j in range(word_length-1):
        process_state_and_embed(states,embeds ,hiddens,outputs,j)

    outputs[-1] = states[-1] @ w_so + b_o
    
    distances = (torch.cdist(outputs,all_char_embeds[:-1]) ** 2).sum(2,keepdim=True)#is actually distances squared
        
    filtered = torch.where(embeds != 1e10, ((outputs-embeds)**2 / distances), 0)
    
    ones = torch.where(embeds != 1e10, 1, 0)
    
    loss = (filtered*adjustment_factors).sum()
    
    loss.backward()
    
    for p in network_params:
        p.data -= 3e-3*p.grad
        p.grad = None
        
    loss_tot+= loss/ adjustment_factors.sum()
    if i % check_every==0 and i>0:
        print(f"{i} of {loops}: " , loss_tot / (check_every))
        loss_tot = 0
#     print('o',outputs)
#     print('e',embeds)
#     print('wl',word_length)

500 of 100000:  tensor(0.0357, dtype=torch.float64, grad_fn=<DivBackward0>)
1000 of 100000:  tensor(0.0345, dtype=torch.float64, grad_fn=<DivBackward0>)
1500 of 100000:  tensor(0.0327, dtype=torch.float64, grad_fn=<DivBackward0>)
2000 of 100000:  tensor(0.0318, dtype=torch.float64, grad_fn=<DivBackward0>)
2500 of 100000:  tensor(0.0307, dtype=torch.float64, grad_fn=<DivBackward0>)
3000 of 100000:  tensor(0.0301, dtype=torch.float64, grad_fn=<DivBackward0>)
3500 of 100000:  tensor(0.0295, dtype=torch.float64, grad_fn=<DivBackward0>)
4000 of 100000:  tensor(0.0289, dtype=torch.float64, grad_fn=<DivBackward0>)
4500 of 100000:  tensor(0.0288, dtype=torch.float64, grad_fn=<DivBackward0>)
5000 of 100000:  tensor(0.0283, dtype=torch.float64, grad_fn=<DivBackward0>)
5500 of 100000:  tensor(0.0276, dtype=torch.float64, grad_fn=<DivBackward0>)
6000 of 100000:  tensor(0.0274, dtype=torch.float64, grad_fn=<DivBackward0>)
6500 of 100000:  tensor(0.0273, dtype=torch.float64, grad_fn=<DivBackward0>)


53500 of 100000:  tensor(0.0165, dtype=torch.float64, grad_fn=<DivBackward0>)
54000 of 100000:  tensor(0.0165, dtype=torch.float64, grad_fn=<DivBackward0>)
54500 of 100000:  tensor(0.0166, dtype=torch.float64, grad_fn=<DivBackward0>)
55000 of 100000:  tensor(0.0162, dtype=torch.float64, grad_fn=<DivBackward0>)
55500 of 100000:  tensor(0.0159, dtype=torch.float64, grad_fn=<DivBackward0>)
56000 of 100000:  tensor(0.0158, dtype=torch.float64, grad_fn=<DivBackward0>)
56500 of 100000:  tensor(0.0160, dtype=torch.float64, grad_fn=<DivBackward0>)
57000 of 100000:  tensor(0.0161, dtype=torch.float64, grad_fn=<DivBackward0>)
57500 of 100000:  tensor(0.0162, dtype=torch.float64, grad_fn=<DivBackward0>)
58000 of 100000:  tensor(0.0157, dtype=torch.float64, grad_fn=<DivBackward0>)
58500 of 100000:  tensor(0.0162, dtype=torch.float64, grad_fn=<DivBackward0>)
59000 of 100000:  tensor(0.0158, dtype=torch.float64, grad_fn=<DivBackward0>)
59500 of 100000:  tensor(0.0157, dtype=torch.float64, grad_fn=<D

In [149]:
def predict(word):
    word_length = len(word)

    embeds = all_char_embeds[word]

    states = [init_state]+[torch.zeros((state_size,)) for _ in range(word_length-1)]

    hiddens = [torch.zeros((embed_size,)) for _ in range(word_length-1)]

    outputs = torch.zeros((word_length,embed_size))

    for i in range(word_length-1):
        process_state_and_embed(states,embeds ,hiddens,outputs,i)

    return states[-1] @ w_so + b_o


def get_next(text):
    word = [char_to_int[c] for c in text] +[0]

    pred = predict(word)

    pred = pred.view(1,embed_size)
    dists = torch.cdist(pred,all_char_embeds[:-1])
    dists = dists.view(-1)
    dists = torch.exp(-5*dists)
    dists/=dists.sum()
    sampled = (torch.multinomial(dists,1)).item()
    return int_to_char[sampled]

In [155]:
start_st = 'jac'

for _ in range(100):
    st = start_st
    curr = 'a'
    while curr!='.':
        curr = get_next(st)
        st+=curr
    print(st)


jacvulaewe.
jaccutheludurut.
jacxafa.
jacvut.
jacs.
jack.
jacmeleutenti.
jaczacahuher.
jacguytu.
jacguubavotetuuwo.
jacve.
jacbatueshnt.
jacpazuweeuces.
jacku.
jacpazutuutaua.
jacfutuerene.
jacfut.
jackuunuqhh.
jacku.
jacxecoderus.
jackuulaqequ.
jacbuus.
jacfu.
jacqu.
jacvaswovoleeates.
jaczaca.
jaczaca.
jacq.
jacpoweh.
jaczagheaubut.
jacfat.
jacmewosaajep.
jaczuteugutugtruxere.
jacb.
jacbuus.
jaczunto.
jacguucopu.
jacvadtetub.
jacvulaewe.
jacfuleetea.
jacxopereenhuwizofut.
jacguafe.
jacmewetorepetnat.
jacfu.
jacquuwes.
jack.
jackuuti.
jacxaxhe.
jacvor.
jacfuraetosurlh.
jacv.
jaczacelu.
jacguaxuteunuutowebenfucothe.
jacguuzebuw.
jacfutuenaturorae.
jackuutiuteteutetuturuc.
jacpaz.
jaccaw.
jacku.
jaczuteupera.
jaccar.
jacfulaotore.
jaczafa.
jacxesherrojireubua.
jacfadhodisa.
jacguuzevose.
jacxes.
jacbuytu.
jacvot.
jacguujek.
jacpet.
jacvadna.
jacf.
jacpazulae.
jacbu.
jaccat.
jacputu.
jacb.
jacmos.
jacguupejelaeleeri.
jacfh.
jacfulaesemola.
jacbuus.
jacxeba.
jacfutueleteadewe.
jaccal.
jac

In [64]:
((all_char_embeds[char_to_int['a']]-all_char_embeds[char_to_int['e']])**2).sum()

tensor(7.9190e-05, grad_fn=<SumBackward0>)

In [151]:
all_char_embeds

tensor([[ 1.1277e+00, -8.8232e-02, -1.0335e+00,  2.5983e+00, -8.8906e-01,
         -3.1492e-01],
        [ 2.2695e+00, -6.4594e-03, -2.1022e+00,  2.8194e+00, -1.0321e+00,
         -1.0548e+00],
        [-1.7191e+00, -4.8078e-01,  9.0666e-01, -1.0642e+00,  6.0315e-01,
          5.1761e-01],
        [-1.3007e+00, -2.9655e-01,  9.0441e-01, -9.7577e-01,  4.4994e-01,
          8.3598e-01],
        [-1.0109e+00, -2.5439e-01,  4.1026e-01, -1.2056e-02,  1.1623e-01,
          1.1777e-01],
        [ 2.7456e+00, -5.0969e-01, -1.8866e+00,  4.1694e+00, -1.5524e+00,
         -8.5540e-01],
        [-1.8919e+00, -1.2437e-01,  1.0180e+00, -1.5503e+00,  7.2874e-01,
          9.2309e-01],
        [-1.4867e+00, -3.3611e-01,  1.0614e+00, -7.1992e-01,  8.0783e-01,
          7.6978e-01],
        [ 1.4528e+00, -6.7225e-01, -1.2368e+00,  2.5108e+00, -2.3421e-01,
         -1.0520e+00],
        [ 3.5377e+00, -8.7898e-01, -1.8128e+00,  4.1241e+00, -1.3459e+00,
         -1.4490e-01],
        [-1.3747e+00, -3.9754e